# Processamento do Arquivo de Vacinação - Recente

**Processamento dos arquivos a partir de 29/05**


Este documento é utilizado para processar o arquivo **XLSX** disponibilizado pela **Secretaria de Estado da Saúde do estado de Sergipe** com os quantitativos da vacinação do estado. Foi utilizada a biblioteca **[pandas](https://pandas.pydata.org/)** para carregar e processar o arquivo com a finalidade de gerar um arquivo .csv mais adequado para o processo de visualização de dados. Um exemplo de visualização destes dados está no Painel COVID-19 disponível [neste link](https://covid19.data2learning.com).

Diferentes tipos de arquivos já foram compartilhados pela secretaria. Esse processamento se refere ao arquivo disponibilizado a partir de **29/05/21**. No dia 29/05 teve uma nova mudança no arquivo do Excel com a inclusão de novas colunas referente a novos grupos de vacionados: Trabalhadores do sistema prisional,Privados de liberdade, Trabalhadores do Transporte aéreo, Trabalhadores da Limpeza Urbana,Trabalhadores do Transporte Coletivo.

**Atenção**

A coluna **Outros** que contabiliza **a população vacinada com resíduos da vacina aos final do dia** foi retirada da tabela a partir do dia 29/05.

**Devida a mudanças constantes em nomes e ordem das tabelas, esse arquivo vai sempre referenciar o mais recente e ficou enviável de manter cópias documentadas de cada alteração que acontece nos documentos. De qualquer forma, vou manter uma lista das alterações que faço para cada arquivo, mas vou manter o código apenas do mais recente.** 

* Dia 30/05: não foram adicionados grupos, mas a ordem das colunas mudaram de acordo com o documento do dia 29/05.
* Dia 31/05: não foram adicionados grupos, mas mudou-se novamente a ordem das colunas. 
* Dia 02/06: Inclusão do grupo caminhoneiros.
* Dia 03/06: Inclusão do grupo 55-59 anos
* Dia 04/06: Inclusão dos grupos: Trabalhadores da Educação Superior, Trabalhadores da Zona Portuária, Trabalhadores Industriais e Civil. Adição da coluna da segunda dose para educação e para população de 55-59 anos.
* Dia 08/06: Inclusão do grupo 50-54. Além disso, mudou ordem das colunas. A coluna 55-59 trocou de lugar coom a de transport aéreo.
* Dia 09/06: Inclusão do grupo 45-49. 
* Dia 10/06: Mudança de ordem da coluna Transporte Aéreo.
* Dia 16/06: Inclusão do grupo 40-44


## Etapas do Processamento - EXCEL

Para trabalhar melhor com as informações, foi preciso fazer um organização das informações disponbilizadas. Apesar do pandas lê facilmente arquivos do Excel, a estrutura em níveis das colunas no documento dificulta um pouco o trabalho no DataFrame gerado. Além disso, o documento possui algumas linhas com informações de título do documento como descrição do Governo, secretaria e coordenação responsável pelo documento. Como a idéia é gerar um arquivo .CSV que seja de fácil leitura, essas informações precisam ser removidas do documento.

As etapas do processamento foram: 

1. Definir uma nomeclatura para cada coluna para excluir a hierarquia criada no arquivo excel. 
2. Definir as colunas de interesse para geração do CSV. 
3. Carregar o XLSX e gerar o arquivo final.

Segue a execução das etapas. 

In [51]:
# Import da biblioteca pandas

import pandas as pd
import numpy as np

### Nomeclatura das colunas

Algumas colunas forma renomeadas de acordo com os grupos de vacinação aptos para vacinação, conforme o arquivo do excel disponibilizado. São eles:

* Grupo 1: Trabalhadores da Saúde
* Grupo 2: Pessoas com 60 anos ou mais e deficientes Institucionalizados
* Grupo 3: Índio Aldeado
* Grupo 4: Pessoas com 80 anos e mais
* Grupo 5: Pessoas de 70 a 79 anos 
* Grupo 6: Pessoas de 60 a 69 anos
* Grupo 7: Situação de Rua
* Grupo 8: Quilombolas
* Grupo 9: Trabalhadores das forças de segurança
* Grupo 10: Pessoas com Síndrome de Dowm e Autismo
* Grupo 11: Outros (População vacinada com resíduos de vacinas ao final do dia) 
* Grupo 12: Puerperas com comorbidade
* Grupo 13: Gestante com comorbidade
* Grupo 14: Deficiente Permanente
* Grupo 15: Comorbidades
* Grupo 16: Trabalhadores da Educação
* Grupo 17: Trabalhadores do sistema prisional
* Grupo 18: Privados de liberdade
* Grupo 19: Trabalhadores do Transporte aéreo
* Grupo 20: Trabalhadores da Limpeza Urbana
* Grupo 21: Trabalhadores do Transporte Coletivo
* Grupo 22: Caminhoneiros
* Grupo 23: População de 55 a 59 anos
* Grupo 24: Trabalhadores da Educação Superior
* Grupo 25: Trabalhadores da Zona Portuária
* Grupo 26: Trabalhadores Industriais e Civil
* Grupo 27: População de 50 a 54 anos
* Grupo 28: População de 45 a 49 anos
* Grupo 29: População de 40 a 44 anos

Com isso foram definidos  os nomes de cada coluna de acordo com o que estava disponível no arquivo original para cada grupo. Além disso, o arquivo contém informações  sobre a estimativa populacional de algunas grupos, os totais de doses aplicadas e a cobertura vacinal. 

In [52]:
# Use esse a partir do dia 30/05
# Lista com os nomes das colunas do documento excel.
'''
Grupo 11 retirado da planilha no dia 29/05
    'aplicadas_dose1_grupo11',
    'aplicadas_dose2_grupo11',

'''

list_columns = [
    'municipio',
    'populacao_total',
    'populacao_grupo6_6069',
    'populacao_grupo5_7079',
    'populacao_grupo4_80mais',
    'enviadas_dose1',
    'enviadas_dose2',
    'aplicadas_dose1',
    'aplicadas_dose2',
    'aplicadas_dose1_grupo1_saude',
    'aplicadas_dose2_grupo1_saude',
    'aplicadas_dose1_grupo2_institucionalizados',
    'aplicadas_dose2_grupo2_institucionalizados',
    'aplicadas_dose1_grupo3_indigenasaldeados',
    'aplicadas_dose2_grupo3_indigenasaldeados',
    'aplicadas_dose1_grupo8_quilombolas',
    'aplicadas_dose2_grupo8_quilombolas',
    'aplicadas_dose1_grupo9_seguranca',
    'aplicadas_dose2_grupo9_seguranca',
    'aplicadas_dose1_grupo10_downautismo',
    'aplicadas_dose2_grupo10_downautismo',
    'aplicadas_dose1_grupo6_6069',
    'aplicadas_dose2_grupo6_6069',
    'aplicadas_dose1_grupo5_7079',
    'aplicadas_dose2_grupo5_7079',
    'aplicadas_dose1_grupo4_80mais',
    'aplicadas_dose2_grupo4_80mais',
    'aplicadas_dose1_grupo12_puerperascomorbidade',
    'aplicadas_dose2_grupo12_puerperascomorbidade',
    'aplicadas_dose1_grupo13_gestantecomorbidade',
    'aplicadas_dose2_grupo13_gestantecomorbidade',
    'aplicadas_dose1_grupo14_deficiente',
    'aplicadas_dose2_grupo14_deficiente',
    'aplicadas_dose1_grupo15_comorbidade',
    'aplicadas_dose2_grupo15_comorbidade',
    'aplicadas_dose1_grupo16_educacao',
    'aplicadas_dose2_grupo16_educacao',
    'aplicadas_dose1_grupo24_educacaosuperior',
    'aplicadas_dose2_grupo24_educacaosuperior',
    'aplicadas_dose1_grupo17_sistemaprisional',
    'aplicadas_dose2_grupo17_sistemaprisional',
    'aplicadas_dose1_grupo18_privadosliberdade',
    'aplicadas_dose2_grupo18_privadosliberdade',
    'aplicadas_dose1_grupo21_transportecoletivo',
    'aplicadas_dose2_grupo21_transportecoletivo',
    'aplicadas_dose1_grupo20_limpezaurbana',
    'aplicadas_dose2_grupo20_limpezaurbana',
    'aplicadas_dose1_grupo7_situacaoderua',
    'aplicadas_dose2_grupo7_situacaoderua',
    'aplicadas_dose1_grupo22_caminhoneiros',
    'aplicadas_dose2_grupo22_caminhoneiros',
    'aplicadas_dose1_grupo25_zonaportuaria',
    'aplicadas_dose2_grupo25_zonaportuaria',
    'aplicadas_dose1_grupo26_industriaiscivil',
    'aplicadas_dose2_grupo26_industriaiscivil',
    'aplicadas_dose1_grupo23_5559',
    'aplicadas_dose2_grupo23_5559',
    'aplicadas_dose1_grupo27_5054',
    'aplicadas_dose2_grupo27_5054',
    'aplicadas_dose1_grupo28_4549',
    'aplicadas_dose2_grupo28_4549',
    'aplicadas_dose1_grupo29_4044',
    'aplicadas_dose2_grupo29_4044',
    'aplicadas_dose1_grupo19_transporteaereo',
    'aplicadas_dose2_grupo19_transporteaereo',
    'cobertura_dose1_populacao_geral',
    'cobertura_dose1_grupo6_6069',
    'cobertura_dose1_grupo5_7079',
    'cobertura_dose1_grupo4_80mais',
    'cobertura_dose1',
    'cobertura_dose2_populacao_geral',
    'cobertura_dose2_grupo6_6069',
    'cobertura_dose2_grupo5_7079',
    'cobertura_dose2_grupo4_80mais',
    'cobertura_dose2'
]

In [53]:
len(list_columns)

75

### Colunas de interesse 

Nem todas as colunas foram incluídas no CSV final. As colunas referente a cobertura possui informação em %. Em alguns testes que foram feitos, o pandas não gerava essa coluna de forma adequada quando era carregado. Como essa é uma informação fácil de calcular pela relação entre as doses envidas e aplicadas, foi preferido retira-la do documento final. No processo de visualização essa informação pode ser facilmente calculada e exibida. Isso é feito no Painel linkado no início do documento. A linha com o total do estado de Sergipe também foi retirada já que isso pode ser calculado a partir dos dados das colunas anteriores.

In [54]:
# Colunas definidas para o documento final

final_columns = [
    'codigo_municipio',
    'municipio',
    'populacao_total',
    'populacao_grupo4_80mais',
    'populacao_grupo5_7079',
    'populacao_grupo6_6069',
    'enviadas_dose1',
    'enviadas_dose2',
    'aplicadas_dose1',
    'aplicadas_dose2',
    'aplicadas_dose1_grupo1_saude',
    'aplicadas_dose1_grupo2_institucionalizados',
    'aplicadas_dose1_grupo3_indigenasaldeados',
    'aplicadas_dose1_grupo4_80mais',
    'aplicadas_dose1_grupo5_7079',
    'aplicadas_dose1_grupo6_6069',
    'aplicadas_dose1_grupo7_situacaoderua',    
    'aplicadas_dose1_grupo8_quilombolas',    
    'aplicadas_dose1_grupo9_seguranca',    
    'aplicadas_dose1_grupo10_downautismo',
    'aplicadas_dose1_grupo12_puerperascomorbidade',
    'aplicadas_dose1_grupo13_gestantecomorbidade',    
    'aplicadas_dose1_grupo14_deficiente',
    'aplicadas_dose1_grupo15_comorbidade',
    'aplicadas_dose1_grupo16_educacao',
    'aplicadas_dose1_grupo17_sistemaprisional',
    'aplicadas_dose1_grupo18_privadosliberdade',
    'aplicadas_dose1_grupo19_transporteaereo',
    'aplicadas_dose1_grupo20_limpezaurbana',
    'aplicadas_dose1_grupo21_transportecoletivo',
    'aplicadas_dose1_grupo22_caminhoneiros',
    'aplicadas_dose1_grupo23_5559',
    'aplicadas_dose1_grupo24_educacaosuperior',
    'aplicadas_dose1_grupo25_zonaportuaria',
    'aplicadas_dose1_grupo26_industriaiscivil',
    'aplicadas_dose1_grupo27_5054',
    'aplicadas_dose1_grupo28_4549',
    'aplicadas_dose1_grupo29_4044',
    'aplicadas_dose2_grupo1_saude',
    'aplicadas_dose2_grupo2_institucionalizados',
    'aplicadas_dose2_grupo3_indigenasaldeados',
    'aplicadas_dose2_grupo4_80mais',
    'aplicadas_dose2_grupo5_7079',
    'aplicadas_dose2_grupo6_6069',
    'aplicadas_dose2_grupo7_situacaoderua',
    'aplicadas_dose2_grupo8_quilombolas',
    'aplicadas_dose2_grupo9_seguranca',
    'aplicadas_dose2_grupo10_downautismo',
    'aplicadas_dose2_grupo12_puerperascomorbidade',
    'aplicadas_dose2_grupo13_gestantecomorbidade',
    'aplicadas_dose2_grupo14_deficiente',
    'aplicadas_dose2_grupo15_comorbidade',
    'aplicadas_dose2_grupo16_educacao',
    'aplicadas_dose2_grupo17_sistemaprisional',
    'aplicadas_dose2_grupo18_privadosliberdade',
    'aplicadas_dose2_grupo19_transporteaereo',
    'aplicadas_dose2_grupo20_limpezaurbana',
    'aplicadas_dose2_grupo21_transportecoletivo',
    'aplicadas_dose2_grupo22_caminhoneiros',
    'aplicadas_dose2_grupo23_5559',
    'aplicadas_dose2_grupo24_educacaosuperior',
    'aplicadas_dose2_grupo25_zonaportuaria',
    'aplicadas_dose2_grupo26_industriaiscivil',
    'aplicadas_dose2_grupo27_5054',
    'aplicadas_dose2_grupo28_4549',
    'aplicadas_dose2_grupo29_4044',
]

### Carregar o XLSX e gerar o arquivo final

Os trechos de códigos a seguir servem para carregar o arquivo .xslx e gerar o arquivo CSV final. Como já definimos os nomes das colunas manualmente, podemos descartar as 3 primeiras linhas do documento que possui informações de título e nome das colunas de forma hierárquica. A explicação de cada etapa está nos comentários. 

Foi necessário aplicar a separação das informações na coluna Município, já que é passado código e nome do município em uma só colunda. No documento final serão geradas duas colunas distintas para essa informação: codigo_municipio e municipio.

In [55]:
data_ = "2021-06-17"
data_temp = data_.replace("-","")

'''
Carrega o arquivo .xlsx descartando as 3 primeiras linhas (skiprows=8) que correspondem
as linhas de informações títulos das colunas. O parâmetro names força que 
o dataframe gerado tenha os nomes das colunas definidas manualmente. 
''' 
data_final = pd.read_excel("data/" + data_ + ".xlsx", skiprows=2,names=list_columns)

'''
.head(-1) exclui a última linha do documento que possui o total de Sergipe. Essa informação
foi retirada porque é facilmente calcula a partir da soma de cada coluna. Isso pode ser feito
no momento de gerar as visualizações

Use -10 de 11/03 até 23/03
Use -9 a partir de 23/03
Use -11 a partir de 29/03
Use -12 em 16/04
Use -11 a partir em 17/04
USE -10 a partir de 20/04
USE -11 em 24/04
USE -10 a partir de 25/04
USE -12 em 31/05
Use -10 a partir de 01/06

''' 
data_final = data_final.head(-10)

'''
Separando a coluna município em duas novas colunas: codigo_municipio e municipio
'''
data_final['codigo_municipio'] = data_final['municipio'].str[0:6]
data_final['municipio'] = data_final['municipio'].str[7:]


'''
Gera o datframe com as colunas selecinadas para o documento final 
'''
data_final = data_final[final_columns]

'''
Alguns municípios possui valores em branco para algumas colunas. Quando o dataframe é carregado,
esses valores são reconhecidos como NULOS. O código a seguir substitui por 0.
'''
data_final = data_final.fillna(0)

'''
Força que as colunas que possui informações da vacinação seja do tipo inteiro.
final_columns[1:] pega da segunda coluna até a última, que são as colunas do tipo inteiro. 
'''
data_final[final_columns[2:]] = data_final[final_columns[2:]].round().astype(int)

'''
Visualiza o dataframe final 
''' 
data_final

,codigo_municipio,municipio,populacao_total,populacao_grupo4_80mais,populacao_grupo5_7079,populacao_grupo6_6069,enviadas_dose1,enviadas_dose2,aplicadas_dose1,aplicadas_dose2,...,aplicadas_dose2_grupo20_limpezaurbana,aplicadas_dose2_grupo21_transportecoletivo,aplicadas_dose2_grupo22_caminhoneiros,aplicadas_dose2_grupo23_5559,aplicadas_dose2_grupo24_educacaosuperior,aplicadas_dose2_grupo25_zonaportuaria,aplicadas_dose2_grupo26_industriaiscivil,aplicadas_dose2_grupo27_5054,aplicadas_dose2_grupo28_4549,aplicadas_dose2_grupo29_4044
0,280010,Amparo de São Francisco,2380,51,89,140,950,306,824,238,...,0,0,0,0,0,0,0,0,0,0
1,280020,Aquidabã,21681,481,932,1543,6808,2964,5868,2376,...,0,0,0,0,0,0,0,0,0,0
2,280030,Aracaju,664908,10149,24138,47854,233969,93970,197743,66303,...,0,0,0,0,0,0,0,0,0,0
3,280040,Arauá,9947,149,348,644,2715,1082,2471,1077,...,0,0,0,0,0,0,0,0,0,0
4,280050,Areia Branca,18686,281,548,1047,5400,1701,5223,1519,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,280720,Siriri,8970,154,316,533,2450,973,2369,915,...,0,0,0,0,0,0,0,0,0,0
71,280730,Telha,3249,49,100,178,882,369,900,367,...,0,0,0,0,0,0,0,0,0,0
72,280740,Tobias Barreto,52530,1046,2188,3513,16781,6538,15594,5952,...,0,0,0,0,0,0,0,0,0,0
73,280750,Tomar do Geru,13535,208,446,760,3309,1336,3038,1146,...,0,0,0,0,0,0,0,0,0,0


O código a seguir gera o arquivo final no formato CSV. O atributo `index=False` força que o documento não tenha um índice por linha. Essa informação, neste caso, não é necessária para os processos de visualização.

In [56]:
data_final.to_csv("output/relatorio_vacina_sergipe.csv", index=False)
data_final.to_csv("output/historico/" + data_temp + ".csv", index=False)

Desenvolvido por **@adolfoguimaraes**.

Qualquer dúvida sobre o código, só entrar em contato: 
* Twitter: https://twitter.com/adolfoguimaraes
* Instagram: https://instagram.com/profadolfoguimaraes
* GitHub: https://github.com/adolfoguimaraes

*Atenção: Esse trabalho é feito de forma totalmente independente e não tem nenhuma ligação com a secretaria que disponibiliza os dados.*